In [10]:
from asteroid.models import BaseModel
import soundfile as sf
import time
from pystoi import stoi
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np

## 计算STOI

In [12]:
mixture_dir = '/home/oneran/Libri2Mix/wav16k/min/dev/mix_both'
s1 = '/home/oneran/Libri2Mix/wav16k/min/dev/s1'
s2 = '/home/oneran/Libri2Mix/wav16k/min/dev/s2'
model = BaseModel.from_pretrained("JorisCos/ConvTasNet_Libri2Mix_sepnoisy_16k")
model.sample_rate = 16000
historys = []
print('load')

def f(x, y):
    if len(x) > len(y):
        return x[:-(len(x) - len(y))]
    else:
        return np.pad(x, ((0, len(y.flatten()) - len(x.flatten()))), mode='constant', constant_values=0)

for file in os.listdir(mixture_dir):
    mixture = mixture_dir + '/' + file
    s1_path = s1 + '/' + file
    s2_path = s2 + '/' + file
    wav1, _ = sf.read(s1_path)
    wav2, _ = sf.read(s2_path)
    wavmix, _ = sf.read(mixture)
    # wavmix = librosa.resample(wavmix, 16000, 8000)
    wavmix = np.expand_dims(wavmix, axis=0).astype(np.float32)
    sep = model.separate(wavmix)
    sep1, sep2 = sep[0, 0], sep[0, 1]
    # sep1, sep2 = f(librosa.resample(sep1, 8000, 16000), wav1), f(librosa.resample(sep2, 8000, 16000), wav2)
    stoi_1_1, stoi_1_2 = stoi(wav1, sep1, 16000), stoi(wav2, sep2, 16000)
    stoi_2_1, stoi_2_2 = stoi(wav2, sep1, 16000), stoi(wav1, sep2, 16000)
    stoi_mean_1, stoi_mean_2 = sum([stoi_1_1, stoi_1_2]), sum([stoi_2_1, stoi_2_2])
    historys.append([stoi_1_1, stoi_1_2] if stoi_mean_1 > stoi_mean_2 else [stoi_2_1, stoi_2_2])    

/home/oneran/.local/lib/python3.10/site-packages/asteroid/models/base_models.py:55: UserWarning: Other sub-components of the model might have a `sample_rate` attribute, be sure to modify them for consistency.
  warnings.warn(


load


KeyboardInterrupt: 

In [48]:
np.mean(historys)

0.8523906486172697

## 测试silentspeaker

In [22]:
import torch
from asteroid.models import ConvTasNet
from asteroid.engine.system import System
from asteroid.engine.optimizers import make_optimizer
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
# torch.set_default_dtype(torch.float16)
checkpoint = torch.load('/home/oneran/Downloads/16k_ep7.ckpt', map_location=torch.device('cpu'))
model = ConvTasNet(2, mask_act='relu', norm_type='gLN', kernel_size=32, stride=16)
model.sample_rate = 16000
model.eval()
temp = checkpoint['state_dict']
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")
optimizer = make_optimizer(model.parameters())
system = System(model, optimizer, loss_func, None)
system.load_state_dict(temp)
system.cpu()
to_save = system.model.serialize()
model.load_state_dict(to_save['state_dict'])

<All keys matched successfully>

In [23]:
model.cuda()

ConvTasNet(
  (encoder): Encoder(
    (filterbank): FreeFB()
  )
  (masker): TDConvNet(
    (bottleneck): Sequential(
      (0): GlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): GlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), gro

In [28]:
import soundfile as sf
import numpy as np
mixture_dir = '/home/oneran/Data/Libri2Mix/wav16k/both/test/'
def energy_level(wav):
    phase_over_wav = wav[np.where(np.abs(wav) >= 0.1)]
    return 10 * np.log10(np.sum(phase_over_wav ** 2) / len(wav) * 16000 + 1e-5)

s1_history = []
s2_history = []
for file in os.listdir(mixture_dir + '/mix_both'):
    # if len(file.split('_')) < 3:
    #     continue
    mixture = mixture_dir + '/mix_both/' + file
    wavmix, _ = sf.read(mixture)
    # wavmix = librosa.resample(wavmix, 16000, 8000)
    wavmix = np.expand_dims(wavmix, axis=0).astype(np.float32)
    # wavmix = torch.Tensor(mixture, device='cuda')
    sep = model.separate(wavmix)
    s1_history.append(energy_level(sep[0, 0]))
    s2_history.append(energy_level(sep[0, 1]))

In [52]:
counter = 0
pos = 0
threshold = 20
for i, s1, s2 in zip(range(len(s1_history)), s1_history, s2_history):
    counter += 1
    if s1 - s2 >= threshold: 
    # if s1 <= 0 or s2 <= 0:
        pos += 1
            
print(counter, pos)

6000 2741


In [51]:
audios = []
for file in os.listdir(mixture_dir + '/mix_both'):
    # if len(file.split('_')) < 3:
    #     continue
    mixture = mixture_dir + '/mix_both/' + file
    audios.append(mixture)
index = np.where(np.abs(np.array(s1_history) - np.array(s2_history)) >= threshold) 
# index = np.where((np.array(s1_history) <= 5) + (np.array(s2_history) <= 5))
_audios = np.array(audios)
len([elem for elem in (set(_audios.tolist()) - set(_audios[index].tolist())) if len(elem.split('_')) >= 4]) + len([elem for elem in _audios[index] if len(elem.split('_')) <= 3])

125

In [33]:
[elem for elem in (set(_audios.tolist()) - set(_audios[index].tolist())) if len(elem.split('_')) >= 4]

['/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/4970-29093-0019_121-121726-0000_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/5142-36377-0013_3729-6852-0040_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/5683-32879-0004_5105-28240-0022_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/7127-75946-0003_3729-6852-0022_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/3729-6852-0013_237-134500-0002_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/5683-32879-0009_2961-960-0018_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/121-121726-0011_237-126133-0017_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/5142-36377-0009_1284-1180-0007_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/7127-75947-0024_1089-134686-0026_single.wav',
 '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/1580-141084-0010_3570-5696-000

In [34]:
def process(wavpath, savepath):
    mix, _ = sf.read(wavpath)
    mix = np.expand_dims(mix, 0).astype(np.float32)
    mix = np.expand_dims(mix, 0).astype(np.float32)
    seps = model.separate(mix)
    sf.write(savepath + 'mix_est1.wav', seps[0, 0], 16000)
    sf.write(savepath + 'mix_est2.wav', seps[0, 1], 16000)
    print(energy_level(seps[0, 0]), energy_level(seps[0, 1]))

In [120]:
fpath = '/home/oneran/Data/Libri2Mix/wav16k/both/test//mix_both/8463-294825-0019_5639-40744-0038_single.wav'
savepath = '/home/oneran/Desktop/'
process(fpath, savepath)

7.470005422202341 11.921510744746058
